After connecting to database, to make it useful we need to create an views to shop it.

There is 2 ways to create views in django. There is function views and there is class views. It will render templates in views. 

First create a template put inside blog/templates/blog_list_view.html

In [ ]:
<html>
<head><title>List View</title></head>
<body>
  <p>{{data}}</p>
</body>
</html>

Put the following function into blog/views.py

In [ ]:
from django.http import HttpResponse
from django.shortcuts import render


def blog_list_view(requests):
    return render(requests, "blog_list_view.html", {"data": "something"})

Or you do it as a class

In [ ]:
from django.views import View
from django.shortcuts import render


class BlogListView(View):
    def get(self, requests):
        return render(requests, "blog_list_view.html", {"data": "something"})

Before it can be used it need to be in urls.py. You can put it in codingshophouse_blog/

In [ ]:
from django.conf.urls import url
from blog.views import BlogListView

urlpatterns = [
    url(r"^$", BlogListView.as_view(), name="blog-list")
]

now in codingshophouse_blog/urls.py

In [ ]:
from django.conf.urls import url
from django.conf.urls import include
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r"^blog/", include("blog.urls"))
]


A few thing here.You just create a view, define a url to the view. From now on we will use class based views. Because there is a few shortcut. Now let's edit a few thing to make things works

In [ ]:
from django.views import View
from django.shortcuts import render
from blog.models import BlogPost


class BlogListView(View):
    def get(self, requests):
        blogs = BlogPost.objects.all()
        return render(requests, "blog_list_view.html", {"blog": blogs})

Now let's update the template

In [ ]:
<html>
<head><title>List View</title></head>
<body>
  {% for i in blog %}
    <p>{{i.title}}</p>
  {% endfor %}
</body>
</html>

Now let's create a new a new read only view. 

In [ ]:
from django.views import View
from django.shortcuts import render
from blog.models import BlogPost
from django.http import Http404


class BlogView(View):
    def get(self, requests, blog_id):
        try:
            blog = BlogPost.objects.get(id=blog_id)
        except BlogPost.DoesNotExist:
            raise Http404("Does Not Exist")
        return render(requests, "blog_view.html", {"blog": blog})

Then update blog/urls.py

In [ ]:
from django.conf.urls import url
from blog.views import BlogListView
from blog.views import BlogView

urlpatterns = [
    url(r"^view/(?P<blog_id>\d+)$", BlogView.as_view(), name="blog-view"),
    url(r"^$", BlogListView.as_view(), name="blog-list"),
]

Also create the template, blog/templates/blog_view.html

In [ ]:
<html>
<head><title>List View</title></head>
<body>
    <p>{{blog.title}}</p>
    <p>{{blog.content}}</p>
</body>
</html>

Now there is one new thing you learn. You can pass parameter via urls and you can pass it like a function

In [ ]:
<html>
<head><title>List View</title></head>
<body>
    <form action="" method="POST">{% csrf_token %}
        <p><input type="text" name="title"></p>
        <p><textarea name="content"></textarea></p>
        <p><input type="submit" value="submit"></p>
    </form>
</body>
</html>

Now create a create view

In [ ]:
from django.views import View
from django.shortcuts import render
from django.shortcuts import redirect
from blog.models import BlogPost


class BlogCreateView(View):
    def get(self, requests):
        
        return render(requests, "blog_create_view.html")
    
    def post(self, requests):
        form_data = requests.POST
        blog = BlogPost()
        
        blog.title = form_data.get("title")
        blog.content = form_data.get("content")
        blog.save()

        
        return redirect("blog-view", blog_id=blog.id)
        

As usual create the urls

In [ ]:
from django.conf.urls import url
from blog.views import BlogListView
from blog.views import BlogView
from blog.views import BlogCreateView

urlpatterns = [
    url(r"^view/(?P<blog_id>\d+)$", BlogView.as_view(), name="blog-view"),
    url(r"^create$", BlogCreateView.as_view(), name="blog-create"),
    url(r"^$", BlogListView.as_view(), name="blog-list"),
]

Now let's do an edit view. Create blog/templates/blog_edit_view.html

In [ ]:
<html>
<head><title>List View</title></head>
<body>
    <form action="" method="POST">{% csrf_token %}
        <p><input type="text" name="title" value="{{blog.title}}"></p>
        <p><textarea name="content">{{blog.content}}</textarea></p>
        <p><input type="submit" value="submit"></p>
    </form>
</body>
</html>

Now add the views

In [ ]:
from django.views import View
from django.shortcuts import render
from blog.models import BlogPost
from django.http import Http404



class BlogEditView(View):
    def get(self, requests, blog_id):
        try:
            blog = BlogPost.objects.get(id=blog_id)
        except BlogPost.DoesNotExist:
            raise Http404("Does Not Exist")
        return render(requests, "blog_edit_view.html", {"blog": blog})
    
    def post(self, requests, blog_id):
        form_data = requests.POST
        try:
            blog = BlogPost.objects.get(id=blog_id)
        except BlogPost.DoesNotExist:
            raise Http404("Does Not Exist")
        blog.title = form_data.get("title")
        blog.content = form_data.get("content")
        blog.save()
        
        return redirect("blog-edit", blog_id=blog.id)

now add urls

In [ ]:
from django.conf.urls import url
from blog.views import BlogListView
from blog.views import BlogView
from blog.views import BlogCreateView
from blog.views import BlogEditView

urlpatterns = [
    url(r"^view/(?P<blog_id>\d+)$", BlogView.as_view(), name="blog-view"),
    url(r"^create$", BlogCreateView.as_view(), name="blog-create"),
    url(r"^edit/(?P<blog_id>\d+)$", BlogEditView.as_view(), name="blog-edit"),
    url(r"^$", BlogListView.as_view(), name="blog-list"),
]
